## Set-Up of AutoGen, get APIs, necessary imports

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print('Groq API Key loaded successfully')
else:
    print('Groq API Key loading failed, please make sure the .env file exists and the spelling is correct')

if os.getenv("TAVILY_API_KEY"):
    print('Tavily API Key loaded successfully')
else:
    print('Tavily API Key loading failed, please make sure the .env file exists and the spelling is correct')

Groq API Key loaded successfully
Tavily API Key loaded successfully


In [2]:
import re
import httpx
import os

from langchain_groq import ChatGroq
load_dotenv()

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:

# Import Autogen components
from autogen import AssistantAgent, UserProxyAgent,  ConversableAgent,register_function
from typing import TypedDict, List, Annotated
import operator
from typing import TypedDict, List, Tuple
import operator

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
llm_config = {"model": "llama-3.1-70b-versatile", "api_key": os.getenv("GROQ_API_KEY"), "api_type": "groq"}
assistant = ConversableAgent("chatbot", llm_config=llm_config)

In [5]:
import os
import autogen

player_config_list = [
    {
        "model": "llama-3.1-70b-versatile",
        "api_key": os.environ.get("GROQ_API_KEY"),
        "api_type": "groq"
    },
]

## Game Setup


In [6]:
import random

class CodenamesGame:
    def __init__(self, starting_team="red"):
        current_dir = os.getcwd()
        wordlist_path = os.path.join(current_dir, 'wordlist-eng.txt')
        
        with open(wordlist_path, 'r') as file:
            all_words = [word.strip() for word in file.readlines()]
        
        self.words = random.sample(all_words, 25)
        self.colors = ["red"] * 8 + ["blue"] * 8 + ["neutral"] * 7 + ["black"] * 2
        random.shuffle(self.colors)
        self.board = dict(zip(self.words, self.colors))
        self.guessed_words = set()
        self.current_team = starting_team
        self.game_over = False
        self.winner = None
        self.turn_ended = False
        self.clue_given = False

    def switch_team(self):
        self.current_team = "blue" if self.current_team == "red" else "red"
        self.turn_ended = False

    def get_board_state(self, for_spymaster=False):
        if for_spymaster:
            return self.board
        else:
            return {word: color if word in self.guessed_words else "unknown" for word, color in self.board.items()}

    def make_guess(self, word):
        if self.game_over or word not in self.words or word in self.guessed_words:
            return False, "Invalid guess"
        
        self.guessed_words.add(word)
        color = self.board[word]
        
        if color == "black":
            self.game_over = True
            self.winner = "blue" if self.current_team == "red" else "red"
            return False, f"Game over! {self.current_team.capitalize()} team guessed the black word. {self.winner.capitalize()} team wins!"
        elif color == self.current_team:
            if self.check_win_condition():
                return True, f"Correct! The word '{word}' belongs to the {self.current_team} team. {self.winner.capitalize()} team wins!"
            return True, f"Correct! The word '{word}' belongs to the {self.current_team} team."
        else:
            self.turn_ended = True
            self.clue_given = False
            if color == "neutral":
                return False, f"Neutral word. Turn ends for the {self.current_team} team."
            else:
                if self.check_win_condition():
                    return False, f"Incorrect. The word '{word}' belongs to the other team. {self.winner.capitalize()} team wins!"
                return False, f"Incorrect. The word '{word}' belongs to the other team. Turn ends for the {self.current_team} team."

    def check_win_condition(self):
        red_words = sum(1 for color in self.board.values() if color == "red")
        blue_words = sum(1 for color in self.board.values() if color == "blue")
        red_guessed = sum(1 for word in self.guessed_words if self.board[word] == "red")
        blue_guessed = sum(1 for word in self.guessed_words if self.board[word] == "blue")
        
        if red_guessed == red_words:
            self.winner = "red"
            self.game_over = True
            return True
        elif blue_guessed == blue_words:
            self.winner = "blue"
            self.game_over = True
            return True
        return False

    def is_game_over(self):
        return self.game_over

    def get_winner(self):
        return self.winner

    def is_turn_ended(self):
        return self.turn_ended

## Stopping conditions

In [21]:

game = CodenamesGame()
def check_made_move(msg):
    
    if game.is_turn_ended():
        return True
    else:
        return False
    
def check_game_ended(msg):
    
    if game.is_game_over():
        return True
    else:
        return False

## Set up Agents

In [22]:
game = CodenamesGame()


blue_spymaster = autogen.ConversableAgent(
    name="Blue Spymaster",
    system_message="""You are the Blue Spymaster in a game of Codenames. Your task is to 
    give a one-word clue and a number.
    The number represents how many words on the board are related to your clue.
    Call get_available_words(colour = blue) first to get the available words that belong to your team.
    Provide a clue in the format: 'clue_word, number'
    Make sure your clue is not any of the words on the board, and follows Codenames rules.""",
    llm_config={"config_list": player_config_list},
    human_input_mode="NEVER",
    is_termination_msg= check_game_ended,

)
red_spymaster = autogen.ConversableAgent(
    name="Red Spymaster",
    system_message="""You are the Red Spymaster in a game of Codenames. Your task is to 
    give a one-word clue and a number.
    The number represents how many words on the board are related to your clue.
    Call get_available_words(colour = red) first to get the available words that belong to your team.
    Provide a clue in the format: 'clue_word, number'
    Make sure your clue is not any of the words on the board, and follows Codenames rules.""",
    llm_config={"config_list": player_config_list},
    human_input_mode="NEVER",
    is_termination_msg= check_game_ended,


)
red_field_agent = autogen.ConversableAgent(
    name="Red Field Agent",
    system_message="""
    You are a Red Field agent in a game of Codenames. Your Spymaster has given 
    you a clue for a number of words. 
    First call get_grid() to get all words that are available to guess on the game grid.
     Based on the clue, suggest up to the provided number of words that you think are 
    related to it. Never provide more Words than the number. Provide your guesses always as a comma-separated list for example like '\n\nPIRATE, LOCH NESS' but not like '\n\nMy guess is: DEGREE'.
    Then for call make_move(words) with your guessed words 
    """,
    is_termination_msg= check_made_move,
    llm_config={"config_list": player_config_list},
    human_input_mode="NEVER",

)
blue_field_agent = autogen.ConversableAgent(
    name= "Blue Field Agent",
    system_message= """You are a Blue Field agent in a game of Codenames. Your Spymaster has given 
    you a clue for a number of words. 
    First call get_grid() to get all words that are available to guess on the game grid.
    Based on the clue, suggest up to the provided number of words that you think are 
    related to it. Never provide more Words than the number. Provide your guesses always as a comma-separated list for example like '\n\nPIRATE, LOCH NESS' but not like '\n\nMy guess is: DEGREE'.
    Then for call make_move(words) with your guessed words 
    """,
    llm_config= {"config_list": player_config_list},
    human_input_mode="NEVER",
    is_termination_msg= check_made_move,
)

## Set up tool functions

In [23]:

def get_available_words(team: Annotated[str, "The colour of your team"])-> Annotated[str, "A list of available words for associated team on the grid"]:
   # Filter out words that have already been guessed
    available_words = [word for word, color in game.get_board_state(for_spymaster= True).items() if color == team and word not in game.guessed_words]
    game.turn_ended= False
    game.current_team = team
    if not available_words:
        return "pass"  # If no words are available, pass
    return  'Available words for your team are: ' + ', '.join(available_words)

def get_grid() -> Annotated[str, "A list of available words for associated team on the grid"]:
    return {word: color if word in game.guessed_words else "unknown" for word, color in game.board.items()}

def make_move(response: Annotated[dict, "A comma-seperated list of the guessed words"]) -> Annotated[str, "Result of the move."]:
    if isinstance(response, dict) and 'content' in response:
        response_content = response['content']
    else:
        response_content = str(response)
    
    guesses = [word.strip() for word in response_content.split(',') if word.strip() in game.get_board_state(for_spymaster=False)]
    
    
    if not guesses:
        return f"{game.current_team.capitalize()} Field Agent didn't provide any valid guesses."
        
    print(f"{game.current_team.capitalize()} Field Agent's guesses: {', '.join(guesses)}")
        
    for guess in guesses:
            success, message = game.make_guess(guess)
            print(message)
            if not success or game.is_turn_ended():
                break
    game.turn_ended = True
    return guesses
        

## Set up Proxy Agent

In [24]:


grid_proxy = ConversableAgent(
    name="Grid Proxy",
    llm_config=False,
    human_input_mode="NEVER",
)

## Register tools for execution

In [25]:
# Register `get_available_words` for the red and blue spymasters
red_spymaster.register_for_llm(name="get_available_words", description="Get available words for a team.")(get_available_words)
blue_spymaster.register_for_llm(name="get_available_words", description="Get available words for a team.")(get_available_words)

# Register `get_grid` and `make_move` for the red and blue field agents
red_field_agent.register_for_llm(name="get_grid", description="Get grid and words.")(get_grid)
blue_field_agent.register_for_llm(name="get_grid", description="Get grid and words.")(get_grid)

red_field_agent.register_for_llm(name="make_move", description="Call this tool to make a move.")(make_move)
blue_field_agent.register_for_llm(name="make_move", description="Call this tool to make a move.")(make_move)

# Register `get_available_words` and `get_grid` for execution by grid_proxy
grid_proxy.register_for_execution(name="get_available_words")(get_available_words)
grid_proxy.register_for_execution(name="get_grid")(get_grid)
grid_proxy.register_for_execution(name="make_move")(make_move)
red_spymaster.register_for_execution(name="get_grid")(get_grid)
blue_spymaster.register_for_execution(name="get_grid")(get_grid)
red_spymaster.register_for_execution(name="make_move")(make_move)
blue_spymaster.register_for_execution(name="make_move")(make_move)
red_field_agent.register_for_execution(name="get_grid")(get_grid)
blue_field_agent.register_for_execution(name="get_grid")(get_grid)
red_field_agent.register_for_execution(name="make_move")(make_move)
blue_field_agent.register_for_execution(name="make_move")(make_move)



<function __main__.make_move(response: typing.Annotated[dict, 'A comma-seperated list of the guessed words']) -> typing.Annotated[str, 'Result of the move.']>

## Set up game flow

In [26]:
nested_chats = [
          {
            # Red Spymaster provides a clue for Red Field Agent.
            "sender": grid_proxy,
            "recipient": red_spymaster,
            "message": "lets play codenames. Be careful not to include one of the available words in your clue answer message ",
            "summary_method": "last_msg",
            "max_turns": 2,
        },
        {
            # Red Spymaster provides a clue for Red Field Agent.
            "sender": red_spymaster,
            "recipient": red_field_agent,
            "message": """Your Spymaster has given you the above clue and a number of words to guess. 
            First call get_grid() to get all words that are available to guess on the game grid.
            Based on the clue, suggest up to the provided number of words that you think are 
            related to it and are still flagged as unknown on the received grid. Never provide more words than the number. 
            Provide your guesses always as a comma-separated list for example like '\n\nPIRATE, LOCH NESS' but not like '\n\nMy guess is: DEGREE'.
            Then for call make_move(response) with your guessed words""",
            "summary_method": "last_msg",

        },
       
    ]

red_spymaster.register_nested_chats(
    trigger=blue_spymaster,
    chat_queue= nested_chats,
    # Blue Spymaster’s turn triggers when Red completes a move.
    
)
nested_chats= [
          {
            # Blue Spymaster provides a clue for Red Field Agent.
            "sender": grid_proxy,
            "recipient": blue_spymaster,
            "message": "lets play codenames. Be careful not to include one of the available words in your clue answer message ",
            "summary_method": "last_msg",
            "max_turns": 2,
        },
        {
            # Blue Spymaster provides a clue for Blue Field Agent.
            "sender": blue_spymaster,
            "recipient": blue_field_agent,
            "message": """Your Spymaster has given you the above clue and a number of words to guess. 
            First call get_grid() to get all words that are available to guess on the game grid.
            Based on the clue, suggest up to the provided number of words that you think are 
            related to it and are still flagged as unknown on the received grid. Never provide more words than the number. 
            Provide your guesses always as a comma-separated list for example like '\n\nPIRATE, LOCH NESS' but not like '\n\nMy guess is: DEGREE'.
            Then for call make_move(response) with your guessed words""",
            "summary_method": "last_msg",
            
        },
     
    ]
blue_spymaster.register_nested_chats(
    trigger=red_spymaster,
    chat_queue= nested_chats,
      # Red Spymaster’s turn triggers when Blue completes a move.
  
)


## Play Game

In [30]:
game = CodenamesGame()

# Initiate the chat sequence with the Red Spymaster's first move

chat_result = blue_spymaster.initiate_chat(red_spymaster,
    message="Let's play codenames! Your move.",
    max_turns=10,
   
)

Blue Spymaster (to Red Spymaster):

Let's play codenames! Your move.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Grid Proxy (to Red Spymaster):

lets play codenames. Be careful not to include one of the available words in your clue answer message 

--------------------------------------------------------------------------------
Red Spymaster (to Grid Proxy):

***** Suggested tool call (call_jg2a): get_available_words *****
Arguments: 
{"team": "red"}
****************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_available_words...
Grid Proxy (to Red Spymaster):

Grid Proxy (to Red Spymaster):

***** Response from calling tool (call_jg2a) *****
Avail